In [20]:
import os
import nibabel as nib
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
import numpy as np
import pandas as pd

In [21]:
# Define the folder path containing NIfTI files
folder_path = "F:/Image CHD/ImageCHD_dataset/Training Data/"

In [22]:
# Define output file path
output_file = "glcm_features.xlsx"

In [23]:
# Define property names for clarity
property_names = ["Contrast", "Dissimilarity", "Homogeneity", "Energy", "Correlation"]

In [24]:
# List to store data for each slice
data_list = []

In [25]:
# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    # Check if it's a NIfTI file
    if not filename.endswith(".nii.gz"):
        continue

    # Load the image
    img = nib.load(os.path.join(folder_path, filename))
    data = img.get_fdata()

    # Iterate through each slice
    for slice_num in range(data.shape[2]):
        # Extract a single 2D slice from the 3D volume
        slice_data = data[:, :, slice_num]

        # Convert to uint8
        slice_data_uint8 = (slice_data * 255).astype(np.uint8)

        # Calculate GLCM and properties
        glcm = graycomatrix(
            slice_data_uint8, distances=[1], angles=[0], symmetric=True, normed=True
        )
        properties = [
            graycoprops(glcm, prop)[0, 0] for prop in ["contrast", "dissimilarity", "homogeneity", "energy", "correlation"]
        ]

        # Calculate additional properties
        mean_val = np.mean(slice_data_uint8)
        variance_val = np.var(slice_data_uint8)
        entropy_val = shannon_entropy(slice_data_uint8)
        second_moment_val = np.mean(slice_data_uint8 ** 2)

        # Prepare data for each slice
        slice_data = {
            "File Name": filename,
            "Slice Number": slice_num + 1,
            "Mean": mean_val,
            "Variance": variance_val,
            "Entropy": entropy_val,
            "Second Moment": second_moment_val
        }

        # Add GLCM properties
        for idx, prop_name in enumerate(property_names):
            slice_data[prop_name] = properties[idx]

        # Append data to list
        data_list.append(slice_data)

In [26]:
# Check if data_list is non-empty
if not data_list:
    print("No NIfTI files found in the specified directory.")
    exit()

In [27]:
# Create and save the DataFrame
data_frame = pd.DataFrame(data_list)
data_frame.to_excel(output_file, index=False)

print(f"GLCM features saved to {output_file}")

GLCM features saved to glcm_features.xlsx


In [2]:
import os
import nibabel as nib
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
import numpy as np
import pandas as pd


In [3]:
# Define output file path
output_file = "glcm_feature.xlsx"
# Define property names for clarity
property_names = ["Contrast", "Dissimilarity", "Homogeneity", "energy", "Correlation"]


In [4]:
# List to store data for each NIfTI file
data_list = []


In [6]:
# Iterate through all files in the folder
for filename in os.listdir("F:\Image CHD\ImageCHD_dataset\Training Data"):
    # Check if it's a NIfTI file
    if not filename.endswith(".nii.gz"):
        continue

    # Load the image
    img = nib.load(f"F:\Image CHD\ImageCHD_dataset\Training Data/{filename}")
    data = img.get_fdata()

# Flatten the volume into a 2D array
flattened_data = data.reshape((-1, data.shape[-1]))

    # Convert to uint8
flattened_data_uint8 = (flattened_data * 255).astype(np.uint8)

    # Calculate GLCM and properties for the entire volume
glcm = graycomatrix(
        flattened_data_uint8, distances=[1], angles=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], symmetric=True, normed=True
    )
properties = graycoprops(glcm, prop=["Contrast", "Dissimilarity", "Homogeneity", "energy", "Correlation"])

    # Prepare data for each file
file_data = {
        "File Name": filename,
    }

    # Add GLCM properties
for idx, prop_name in enumerate(property_names):
        prop_values = np.array(properties[idx]).reshape(-1)
        for angle, value in zip([0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], prop_values):
            file_data[f"{prop_name} - {int(np.degrees(angle))} deg"] = value

    # Append data to list
data_list.append(file_data)

ValueError: ['Contrast', 'Dissimilarity', 'Homogeneity', 'energy', 'Correlation'] is an invalid property